In [ ]:
!pip install opencv-python ultralytics
!pip install imageio[ffmpeg]

In [ ]:
import cv2
import imageio
import json
from ultralytics import YOLO
from pathlib import Path

# Load an official Segment model from YOLOv9
model = YOLO('yolov9e-seg.pt')


In [ ]:
#  Function for person tracking with json outputs and optional videos with annotation 
def person_tracking(video_path, person_only=True, save_video=True):
    # open the video file
    reader = imageio.get_reader(video_path)
    frames = []
    i = 0
    all_object_data = []
    file_name = Path(video_path).stem

    for frame in reader:
        # Convert frame from RGB (imageio's default) to BGR (OpenCV's default)
        frame_bgr = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
        try:
            # Run YOLOv9 tracking on the frame, persisting tracks between frames with bytetrack
            conf = 0.2
            iou = 0.5
            results = model.track(frame_bgr, persist=True, conf=conf, iou=iou, show=False, tracker="bytetrack.yaml")

            # change detection results to Person pathing API output formats.
            object_json = change_format(results[0], i, person_only)
            all_object_data.append(object_json)

            # Append the annotated frame to the frames list (for mp4 creation)
            annotated_frame = results[0].plot()
            frames.append(annotated_frame)
            i += 1

        except Exception as e:
            print(f"Error processing frame: {e}")
            break

    # save the object tracking array to json file
    with open(f'{file_name}_output.json', 'w') as file:
        json.dump(all_object_data, file, indent=4)

     # save annotated video
    if save_video is True:
        # Create a VideoWriter object of mp4
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        output_path = f"output/{file_name}_annotated.mp4"
        fps = reader.get_meta_data()['fps']
        frame_size = reader.get_meta_data()['size']
        video_writer = cv2.VideoWriter(output_path, fourcc, fps, frame_size)

        # Write each frame to the video and release the video writer object when done
        for frame in frames:
            video_writer.write(frame)
        video_writer.release()
        print(f"Video saved to {output_path}")

    return all_object_data


In [ ]:
# define the function that changes YOLOV9 output to Person pathing API output format
def change_format(results, ts, person_only):
    #set person_only to True if you only want to track persons, not other objects.
    object_json = []

    for i, obj in enumerate(results.boxes):
        x_center, y_center, width, height = obj.xywhn[0]
        # Calculate Left and Top from center
        left = x_center - (width / 2)
        top = y_center - (height / 2)
        obj_name = results.names[int(obj.cls)]
        # Create dictionary for each object detected
        if (person_only and obj_name == "person") or not person_only:
            obj_data = {
                obj_name: {
                    "BoundingBox": {
                        "Height": float(height),
                        "Left": float(left),
                        "Top": float(top),
                        "Width": float(width)
                    },
                    "Index": int(obj.id)  # Object index
                },
                "Timestamp": ts  # timestamp of the detected object
            }
        object_json.append(obj_data)

    return object_json

In [ ]:
#main function to call 
video_path = 'videoblocks-aerial-view-people-walking-down-center-lane-grocery-shopping-by-produce-sec_ryyqlpbp3__9a31ad8ce3f137b6572eefb5e6255d81__P360.mp4'
all_object_data = person_tracking(video_path, person_only=True, save_video=True)